In [1]:
import os
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Model

def get_features(img):

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    avg_features = np.mean(features, axis=(1, 2))
    
    return avg_features
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

casc_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(casc_path)

filepath = "image\\practice\\"
files = os.listdir(filepath)

imgs = []

for file in files:

    savepath = f'output\\{file.replace(".jpg", "")}\\'
    if not os.path.exists(savepath):
        os.makedirs(savepath)    

    img = cv2.imread(filepath+file)
    cv2.imwrite(savepath+"0_original.jpg", img)
    faces = faceCascade.detectMultiScale(img, scaleFactor=1.20, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    imgheight = img.shape[0]
    imgwidth = img.shape[1]
    cv2.rectangle(img, (10, imgheight-20), (110, imgheight), (0, 0, 0), -1)

    count = 1
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (128, 255, 0), 2)

        facename = savepath + f"2_face_{str(count)}.jpg"
        face = img[y: y+h, x: x+w]
        face = cv2.resize(face, (32, 32))
        cv2.imwrite(facename, face)        

        face_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        imgs_rgb = []
        imgs_grey = []

        for i in range(8, 0, -1):
            quantized_image = face_pil.quantize(colors=2**i)

            if i == 3:
                feature = get_features(quantized_image.convert("RGB"))
                np.savetxt(f"{savepath}7_face_{str(count)}_feature.csv", feature)

            quantized_image_rgb = quantized_image.convert("RGB")            
            imgs_rgb.append(quantized_image_rgb)
            rgb_array = np.array(quantized_image_rgb)     

            

            with open(f"{savepath}4_face_{str(count)}_rgb_{str(2**i)}.csv", "w") as f:
                f.write("\t" + "\t".join([str(n) for n in range(rgb_array.shape[1])]) + "\n")
                for ii in range(rgb_array.shape[0]):
                    f.write(f"{str(ii)}\t")
                    for jj in range(rgb_array.shape[1]):
                        line = f"[{rgb_array[ii, jj, 0]}, {rgb_array[ii, jj, 1]}, {rgb_array[ii, jj, 2]}]"
                        if jj < rgb_array.shape[1] - 1:
                            line += "\t"
                        f.write(line)
                    if ii < rgb_array.shape[0] - 1:
                        f.write("\n")

            quantized_image_gray = quantized_image.convert("L")
            imgs_grey.append(quantized_image_gray)
            gray_array = np.array(quantized_image_gray)            
            gray_array = pd.DataFrame(gray_array)
            gray_array.to_csv(f"{savepath}6_face_{str(count)}_gray_{str(2**i)}.csv", sep="\t")

        fig, axes = plt.subplots(2, 4, figsize=(12, 6))

        for i, ax in enumerate(axes.flat):
            ax.imshow(np.array(imgs_rgb[i]))
            ax.axis('off')

        face_gird = savepath + f"3_face_{str(count)}_rgb_grid.jpg"
        plt.savefig(face_gird)

        for i, ax in enumerate(axes.flat):
            ax.imshow(np.array(imgs_grey[i]), cmap="gray")
            ax.axis('off')

        face_gird = savepath + f"5_face_{str(count)}_gray_grid.jpg"
        plt.savefig(face_gird)
        plt.close()

        count+=1

    cv2.imwrite(savepath+"1_detected.jpg", img)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
